* Выполняемое задание -- Задание 3 "Многоклассовая классификация и множественная классификация/регрессия"
* Студент: Шеверев Сергей Вячеславови, 22М-05ММ
* Все пункты обязательной части задания

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# устанавливаем точность чисел с плавающей точкой
%precision %.4f

import warnings
warnings.filterwarnings('ignore')  # отключаем предупреждения

## Dataset
Считаем датасет стоимости автомобилей из Задания 1. Датасет модифицирован, в нем выполнены преобразования данных, описанные в блоткноте задания 1.

In [2]:
df = pd.read_csv('car_price_modified.csv')
df.head()

,id,symboling,drivewheel,enginelocation,enginesize,horsepower,price,manufacturer,mpg,coupe,size
0,0,0.0,2,0,1,111,13495.0,14,24.0,1,5.280199
1,1,0.0,2,0,1,111,16500.0,14,24.0,1,5.280199
2,2,2.0,2,0,2,154,16500.0,14,22.5,1,5.875926
3,3,1.0,1,0,1,102,13950.0,17,27.0,0,6.348170
4,4,1.0,3,0,1,115,17450.0,17,20.0,0,6.367348


Поле "id" --- есть индекс. Избавимся от него

In [3]:
df.drop("id", axis=1, inplace=True)
df

,symboling,drivewheel,enginelocation,enginesize,horsepower,price,manufacturer,mpg,coupe,size
0,0.0,2,0,1,111,13495.0,14,24.0,1,5.280199
1,0.0,2,0,1,111,16500.0,14,24.0,1,5.280199
2,2.0,2,0,2,154,16500.0,14,22.5,1,5.875926
3,1.0,1,0,1,102,13950.0,17,27.0,0,6.348170
4,1.0,3,0,1,115,17450.0,17,20.0,0,6.367348
...,...,...,...,...,...,...,...,...,...,...
174,4.0,2,0,1,114,16845.0,16,25.5,0,7.219618
175,4.0,2,0,1,160,19045.0,16,22.0,0,7.209139
176,4.0,2,0,2,134,21485.0,16,20.5,0,7.219618
177,4.0,2,0,1,106,22470.0,16,26.5,0,7.219618


Признаки:
* symboling -- класс безопасности авто (0 - 5)
* driverwheel описывает, какой привод у авто (полный, задний, передний)
* enginelocation --- расположение двигателя (заднее, переднее)
* enginesize --- класс автомобиля по размеру двигателя (малолитражка, средняя, с большим объемом двигателя)
* horsepower --- количество лошадиных сил 
* price --- цена авто
* manufacturer --- порядковый признак производителя авто
* mpg --- усредненный расход топлива автомобиля
* coupe описывает имеет ли авто кузов купе
* size --- геометрические размеры автомобиля


In [4]:
pd.DataFrame(data=['chevrolet',  'honda',  'dodge',  'plymouth',  'subaru',  'isuzu',  'mitsubishi',   'renault',  'toyota',   'volkswagen',  'mazda',  'nissan',  'saab',  'peugeot','alfa-romero',  'mercury',  'volvo',  'audi',  'bmw',  'porsche',  'buick',  'jaguar'])


,0
0,chevrolet
1,honda
2,dodge
3,plymouth
4,subaru
5,isuzu
6,mitsubishi
7,renault
8,toyota
9,volkswagen


Выполним one-hot encoding:

In [5]:
cat_features = ['drivewheel', 'enginesize', 'symboling']
df = pd.get_dummies(df, prefix=cat_features, columns=cat_features)

Для решения задачи "Multiclass classification" требуется наличие более чем двух классов. Поэтому преобразуем количественный признак "price" в категориальный:

In [6]:
df.price.describe()


count      179.000000
mean     13522.732777
std       7871.571293
min       5118.000000
25%       7898.000000
50%      10945.000000
75%      16662.500000
max      45400.000000
Name: price, dtype: float64

In [7]:
to_apply = lambda x: 0 if x <= 7898 else 1 if x  <= 10945 else  2 if x <= 16662.5 else 3
df.price = df.price.apply(to_apply)
df.groupby(by="price").price.count()

price
0    46
1    44
2    44
3    45
Name: price, dtype: int64

Отлично получилось, автомоболей каждого ценового сегмета примерно одинаковое количество.

## Подготовка данных

Разделим данные на тестовую и обучающую выборку, для воспроизводимости результатов зададим random_state, равным 5

In [8]:
from sklearn.model_selection import train_test_split

y = df["price"]
X = df.drop(["price"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

Выполним масштабирование данных:

In [9]:
from sklearn.preprocessing import StandardScaler
features = list(df.keys())
features.remove("price")
numeric_features = ['horsepower', 'manufacturer', 'mpg', 'size']
binary_features = features;
for i in numeric_features:
    binary_features.remove(i)

scaler = StandardScaler()  # воспользуемся стандартным трансформером

# масштабируем обучающую выборку
X_train_scaled = scaler.fit_transform(X_train[numeric_features])

# масштабируем тестовую выборку
X_test_scaled = scaler.transform(X_test[numeric_features])

X_train = np.c_[X_train_scaled, np.array(X_train[binary_features])]
X_test = np.c_[X_test_scaled, np.array(X_test[binary_features])]

## Обучение моделей

Используя кросс-валидацию и подбор гиперпараметров обучим модели (logistic regression, svm, knn, naive bayes, decision tree) с использованием разных стратегий:
* OneVsRest
* OneVsOne
* OutputCode

In [14]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier, OutputCodeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import label_binarize
import math
from time import process_time

def adjust_params(d):
    old_key = list(d.keys())
    new_key = list(map(lambda x: x.replace("estimator__", "").replace("base_", ""), d.keys()))
    for i in range(len(new_key)):
        d[new_key[i]] = d.pop(old_key[i])
    return d

labels = sorted(df.price.unique())
models = [LogisticRegression(), KNeighborsClassifier(), SVC(probability=True), GaussianNB(), DecisionTreeClassifier()]
models_name = ['Logistic regression', 'knn', 'svm', 'naive bayes', 'decision tree']
max_name_len = max([len(i) for i in models_name])
models_params = [{'estimator__C': np.linspace(0.1**3, 3, math.floor(3/0.1))},
                 {'estimator__n_neighbors': range (3, 8), 'estimator__weights': ['uniform', 'distance'], 'estimator__p':[1, 2]},
                 {'estimator__C': np.linspace(0.1**3, 3, math.floor(3/0.1)), 'estimator__kernel':['linear', 'poly', 'rbf','sigmoid'], "estimator__degree":range(1, 6), 'estimator__gamma':['scale', 'auto']},
                 {'estimator__var_smoothing': np.logspace(0,-9, num=100)},
                 {"estimator__criterion": ['gini', 'entropy', 'log_loss'], "estimator__splitter": ['best', 'random'], 'estimator__class_weight':[None, 'balanced']}
                ]

OVR = {"best_models_params":[], "accuracies":[], "times": [], "model": []}
OVO = {"best_models_params":[], "accuracies":[], "times": [], "model": []}
OCC = {"best_models_params":[], "accuracies":[], "times": [], "model": []}

for i in range(len(models)):
    print(models_name[i])
    if type(models[i]) is not DecisionTreeClassifier:
        
        searcher_OVR = GridSearchCV(OneVsRestClassifier(models[i], n_jobs=-1), param_grid = models_params[i], scoring="roc_auc", refit=True,  cv = 5)
        searcher_OVR.fit(X_train, y_train)
        params = adjust_params(searcher_OVR.best_params_)
        time_start = process_time()
        m = models[i].set_params(**params)
        m = OneVsRestClassifier(m)
        m.fit(X_train, y_train)
        time_stop = process_time()
        OVR["times"].append(time_stop - time_start)
        
        prediction = searcher_OVR.predict_proba(X_test)
        OVR["accuracies"].append(roc_auc_score(y_test, prediction,multi_class='ovr'))
        OVR["best_models_params"].append(searcher_OVR.best_params_)
        
        OVR["model"].append(models_name[i])
    else:
        searcher_OVR = GridSearchCV(models[i], param_grid = {"criterion": ['gini', 'entropy', 'log_loss'], "splitter": ['best', 'random'], 'class_weight':[None, 'balanced']}, scoring="roc_auc", refit=True,  cv = 5)
        searcher_OVR.fit(X_train, y_train)
        
        params = adjust_params(searcher_OVR.best_params_)
        time_start = process_time()
        m = models[i].set_params(**params)
        m.fit(X_train, y_train)
        time_stop = process_time()
        OVR["times"].append(time_stop - time_start)
        
        prediction = searcher_OVR.predict_proba(X_test)
        OVR["accuracies"].append(roc_auc_score(y_test, prediction,multi_class='ovr'))
        OVR["best_models_params"].append(searcher_OVR.best_params_)
        OVR["model"].append(models_name[i])
    
    
    searcher_OVO = GridSearchCV(OneVsOneClassifier(models[i], n_jobs=-1), param_grid = models_params[i], scoring="roc_auc", refit=True,  cv = 5)
    params = adjust_params(searcher_OVR.best_params_)
    time_start = process_time()
    m = models[i].set_params(**params)
    m = OneVsOneClassifier(m)
    m.fit(X_train, y_train)
    time_stop = process_time()
    
    searcher_OVO.fit(X_train, y_train)
    
    OVO["times"].append(time_stop - time_start)
    
    prediction = searcher_OVO.predict(X_test)
    prediction = label_binarize(prediction, classes=labels)
    OVO["accuracies"].append(roc_auc_score(y_test, prediction,multi_class='ovo'))
    OVO["best_models_params"].append(searcher_OVO.best_params_)
    OVO["model"].append(models_name[i])
    
    
    searcher_OCC = GridSearchCV(OutputCodeClassifier(models[i], n_jobs=-1), param_grid = models_params[i], scoring="roc_auc", refit=True,  cv = 5)
    searcher_OCC.fit(X_train, y_train)
    params = adjust_params(searcher_OVR.best_params_)
    time_start = process_time()
    m = models[i].set_params(**params)
    m = OutputCodeClassifier(m)
    m.fit(X_train, y_train)
    time_stop = process_time()
    
    
    OCC["times"].append(time_stop - time_start)
    
    prediction = searcher_OCC.predict(X_test)
    prediction = label_binarize(prediction, classes=labels)
    OCC["accuracies"].append(roc_auc_score(y_test, prediction,multi_class='ovo'))
    OCC["best_models_params"].append(searcher_OCC.best_params_)
    OCC["model"].append(models_name[i])


Logistic regression
knn
svm
naive bayes
decision tree


## Результаты обучения

Распечатаем таблицы результатов обучения:

In [15]:
for i in zip([OVR, OVO, OCC], ["OneVsRest results", "OneVsOne results", "OutputCode results"]):
    RES = pd.DataFrame(data=i[0])
    RES.drop("best_models_params", axis=1,  inplace=True)
    print(str(i[1]) + f" Total time is {RES.times.sum():.3f}" + f" Total accuracy is {RES.accuracies.sum():.3f}")
    print(RES, end="\n\n")

OneVsRest results Total time is 0.019 Total accuracy is 4.414
   accuracies     times                model
0    0.810036  0.006940  Logistic regression
1    0.898437  0.001874                  knn
2    0.915348  0.007539                  svm
3    0.927295  0.002422          naive bayes
4    0.863283  0.000461        decision tree

OneVsOne results Total time is 0.053 Total accuracy is 3.510
   accuracies     times                model
0    0.583333  0.008000  Logistic regression
1    0.816667  0.034144                  knn
2    0.500000  0.006104                  svm
3    0.814815  0.002521          naive bayes
4    0.795370  0.002553        decision tree

OutputCode results Total time is 0.039 Total accuracy is 3.504
   accuracies     times                model
0    0.557407  0.016013  Logistic regression
1    0.816667  0.002663                  knn
2    0.500000  0.010359                  svm
3    0.814815  0.003975          naive bayes
4    0.814815  0.006237        decision tree



### Скорость:

1) OneVsRest -- быстрее всех обучается 
2) OutputClassifier --  медленнее чем OneVsRest (практичеески в 2 раза)
3) OneVsOne --  медленнее других стратегий
* \*имеется в виду скорость обучения моделей при применении стратегии

### Точность:
1) OneVsRest -- показывает ощутимо большую точность, нежели другие стратегии
2) OneVsOne -- несколько точнее OutputCode, но незначительно
3) OutputCode -- обладает худшей точностью


* Модели knn, decision tree и naive bayes обладают высокими показателями точности при применении всех стратегий.
* SVM и Logistic Regression - аутсайдеры, при применении этих моделей в стратегиях OneVsOne и outputcode им не удалось показать сопоставимую с другими моделями точности, разве что с logistic regression ее сравнить можно. Судя по всему модели просто угадывают.

## Допольнительный пункт задания (Multilabel Classification)

### Dataset

In [3]:
df = pd.read_csv('semeval2014.csv')
print(df.isnull().sum())
df.head()


text                       0
service                    0
food                       0
anecdotes/miscellaneous    0
price                      0
ambience                   0
dtype: int64


,text,service,food,anecdotes/miscellaneous,price,ambience
0,but the staff was so horrible to us,1,0,0,0,0
1,to be completely fair the only redeeming facto...,0,1,1,0,0
2,the food is uniformly exceptional with a very ...,0,1,0,0,0
3,where gabriela personaly greets you and recomm...,1,0,0,0,0
4,for those that go once and dont enjoy it all i...,0,0,1,0,0


Датасет имеет признак "text", который представлен в виде текста. Работать с текстом напрямую у нас не получится, поэтому необходимо его как-то преобразовать.

Для этого воспользуемся TfidfVectorizer, который реализует технику TF-IDF.  Согласно Википедии, TF-IDF — это техника , используемая для оценки важности слова в контексте документа. Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех остальных документах. 

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X = df["text"]
y = df.drop("text", axis=1)

# initializing TfidfVectorizer
vetorizar = TfidfVectorizer(max_features=3000, max_df=0.85)
# fitting the tf-idf on the given data
vetorizar.fit(X)

# splitting the data to training and testing data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

# transforming the data
X_train = vetorizar.transform(X_train).toarray()
X_test = vetorizar.transform(X_test).toarray()


Теперь текст представлен в виде чисел, с которыми мы уже можем привычно работать.

Предсказывать предстоит значения всех бинарных признаков в датасете, каждый из которых определяет наличие некоторых аспектов этого высказывания.
### Обучение моделей

In [5]:
from sklearn.multioutput import MultiOutputClassifier, ClassifierChain
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier, OutputCodeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import label_binarize
import math
from time import process_time

def adjust_params(d):
    old_key = list(d.keys())
    new_key = list(map(lambda x: x.replace("estimator__", "").replace("base_", ""), d.keys()))
    for i in range(len(new_key)):
        d[new_key[i]] = d.pop(old_key[i])
    return d

models = [LogisticRegression(), KNeighborsClassifier(), SVC(probability=True), GaussianNB(), DecisionTreeClassifier()]
models_name = ['Logistic regression', 'knn', 'svm', 'naive bayes', 'decision tree']

models_params = [{ 'estimator__C': np.linspace(0.1**3, 3, math.floor(3/0.1))},
                 {'estimator__n_neighbors': range (3, 8), 'estimator__weights': ['uniform', 'distance'], 'estimator__p':[1, 2]},
                 {'estimator__kernel':['linear', 'poly', 'rbf','sigmoid'],"estimator__degree":range(2, 5), 'estimator__gamma':['scale', 'auto']},
                 {'estimator__var_smoothing': np.logspace(0,-9, num=100)},
                 {"estimator__criterion": ['gini', 'entropy', 'log_loss'], "estimator__splitter": ['best', 'random'], 'estimator__class_weight':[None, 'balanced']}
                ] #'estimator__C': np.linspace(0.1**3, 3, math.floor(3/0.1)), 
models_params_ = [{ 'base_estimator__C': np.linspace(0.1**3, 3, math.floor(3/0.1))},
                 {'base_estimator__n_neighbors': range (3, 8), 'base_estimator__weights': ['uniform', 'distance'], 'base_estimator__p':[1, 2]},
                 {'base_estimator__kernel':['linear', 'poly', 'rbf','sigmoid'], "base_estimator__degree":range(2, 5),'base_estimator__gamma':['scale', 'auto']},
                 {'base_estimator__var_smoothing': np.logspace(0,-9, num=100)},
                 {"base_estimator__criterion": ['gini', 'entropy', 'log_loss'], "base_estimator__splitter": ['best', 'random'], 'base_estimator__class_weight':[None, 'balanced']}
                ]#'base_estimator__C': np.linspace(0.1**3, 3, math.floor(3/0.1)), 


MOC = {"best_models_params":[], "accuracies":[], "times": [], "model": []}
MOC_ = MOC.copy()
CC = {"best_models_params":[], "accuracies":[], "times": [], "model": []}
CC_ = CC.copy()
for i in range(0,len(models)):
    #if i == 2:
    #    continue
    #print(models_name[i])
    searcher = GridSearchCV(MultiOutputClassifier(models[i]), param_grid = models_params[i], scoring="roc_auc", cv=5, refit=True, n_jobs=-1)
    searcher.fit(X_train, y_train)
    
    params = adjust_params(searcher.best_params_)
    time_start = process_time()
    m = models[i].set_params(**params)
    m = MultiOutputClassifier(m)
    m.fit(X_train, y_train)
    time_stop = process_time()
    
    pred = searcher.predict(X_test)
    MOC["accuracies"].append(roc_auc_score(y_test, pred, multi_class="ovo"))
    MOC["best_models_params"].append(searcher.best_params_)
    MOC["times"].append(-time_start + time_stop)
    MOC["model"].append(models_name[i])
    searcher = GridSearchCV(ClassifierChain(models[i]), param_grid = models_params_[i], scoring="roc_auc", cv=5, refit=True, n_jobs=-1)
    searcher.fit(X_train, y_train)
    
    params = adjust_params(searcher.best_params_)
    time_start = process_time()
    m = models[i].set_params(**params)
    m = ClassifierChain(m)
    m.fit(X_train, y_train)
    time_stop = process_time()
    
    
    pred = searcher.predict(X_test)
    CC["accuracies"].append(roc_auc_score(y_test, pred, multi_class="ovo"))
    CC["best_models_params"].append(searcher.best_params_)
    CC["times"].append(-time_start + time_stop)
    CC["model"].append(models_name[i])

### Результаты обучения
Распечатаем таблицы результатов обучения:

In [6]:
for i in zip([MOC, CC], ["MultiOutPut", "ClassifierChain"]):
    RES = pd.DataFrame(data=i[0])
    RES.drop("best_models_params", axis=1,  inplace=True)
    print(str(i[1]) + f" Total time is {RES.times.sum():.3f}" + f" Total accuracy is {RES.accuracies.sum():.3f}")
    print(RES, end="\n\n")

MultiOutPut Total time is 154.752 Total accuracy is 3.564
   accuracies       times                model
0    0.753446   14.664054  Logistic regression
1    0.570189    0.022103                  knn
2    0.730752  133.394626                  svm
3    0.733893    0.264609          naive bayes
4    0.775998    6.406482        decision tree

ClassifierChain Total time is 105.949 Total accuracy is 3.326
   accuracies      times                model
0    0.754731  17.524426  Logistic regression
1    0.575391   0.068613                  knn
2    0.500000  84.248828                  svm
3    0.714296   0.287025          naive bayes
4    0.781568   3.819825        decision tree



### Скорость обучения
В глаза несколько бросается в разы более длительное обучение моделей SVM относительно остальных моделей при применении обеих стратегий (MultuOutPut и ClassifierChain). Также на фоне остальных моделей выделяется модель логистической регрессии (Logictic regression).

Если не брать в расчет длительность обучения SVM, то в среднем обе стратегии показывают примерно одинаковые скорости обучения.

Посмотрим на разницу во времени обучения конкретных методов:

In [10]:
data = {
    "Model": models_name,
    "time delta, s": [math.fabs(MOC["times"][i] - CC["times"][i]) for i in range(len(models))],
    "time delta, %": [100*math.fabs(MOC["times"][i] - CC["times"][i])/max(MOC["times"][i], CC["times"][i]) for i in range(len(models))],
    "fastest strategy": ["MOC" if MOC["times"][i] < CC["times"][i] else "CC" for i in range(len(models))]
}
pd.DataFrame(data=data)
    

,Model,"time delta, s","time delta, %",fastest strategy
0,Logistic regression,2.860371,16.322198,MOC
1,knn,0.046510,67.786515,MOC
2,svm,49.145798,36.842412,CC
3,naive bayes,0.022415,7.809560,MOC
4,decision tree,2.586657,40.375618,CC


#### Точность предсказаний
Сравнивая метрики AUC-ROC, хочется отметить, что результаты довольно близки, за тем лишь исключением, что SVM при использовании стратегии ClassifierChain  показал себя заметно хуже чем при применении MultiOutPut. Более того, SVM при затраченных временных ресурсах на обучение (не говоря уже о подборе гиперпараметров) кажется наименее привлекательной моделью: она либо работает не лучше оствальных, либо хуже в зависимоти от стратегии.

Относительно других моделей можно сказать следующее: они имеют схожую точность предсказаний при обеих стратегиях. 

Предлагаю посмотреть на аналогичную таблицу для точности:

In [12]:
data = {
    "Model": models_name,
    "accuracy delta": [math.fabs(MOC["accuracies"][i] - CC["accuracies"][i]) for i in range(len(models))],
    "accuracy delta, %": [100*math.fabs(MOC["accuracies"][i] - CC["accuracies"][i])/max(MOC["accuracies"][i], CC["accuracies"][i]) for i in range(len(models))],
    "accuratest strategy": ["MOC" if MOC["accuracies"][i] > CC["accuracies"][i] else "CC" for i in range(len(models))]
}
pd.DataFrame(data=data)

,Model,accuracy delta,"accuracy delta, %",accuratest strategy
0,Logistic regression,0.001285,0.170293,CC
1,knn,0.005202,0.904130,CC
2,svm,0.230752,31.577333,MOC
3,naive bayes,0.019597,2.670342,MOC
4,decision tree,0.005570,0.712650,CC


Видно, что точности моделей отличаются не значительно за исключением SVM.